Case 1 :
 A product-based company has experienced a continuous revenue decline over the past five
 months. The CEO wants a comprehensive diagnostic analysis to uncover the root causes.
 The focus should be on understanding key metrics, evaluating product performance, and
 assessing the contributions and efficiency of each team, including Sales, Marketing, Product
 Development,Branding and Customer Support.
 The CEO also wants insights into external factors, such as market trends and competitor
 dynamics, to determine their role in the revenue decline. The goal is to provide actionable
 recommendations to address the root causes and create a sustainable strategy for recovery.

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [3]:
file_path = r"D:\Projects\Guvi_DA_task\sample_data.csv"
df = pd.read_csv(file_path)

In [4]:
df.columns = [c.strip().replace(" ", "_").lower() for c in df.columns]
df['sales_date'] = pd.to_datetime(df['sales_date'], errors='coerce')
df['lead_registered_time'] = pd.to_datetime(df['lead_registered_time'], errors='coerce')
df['product_amount_with_gst'] = pd.to_numeric(df['product_amount_with_gst'], errors='coerce').fillna(0)
df['user_id'] = df['user_id'].astype(str)
df['coupon_code'] = df['coupon_code'].fillna("NoCoupon")
df['payment_mode'] = df['payment_mode'].fillna("Unknown")
df['transaction_bank'] = df['transaction_bank'].fillna("Unknown")
df['payment_status'] = df['payment_status'].fillna("Unknown")

In [ ]:
df = df[~df['sales_date'].isna()].copy()  
df['sales_month'] = df['sales_date'].dt.to_period('M').dt.to_timestamp()


In [7]:
outdir = "task2_questions_by_step"
os.makedirs(outdir, exist_ok=True)

Solving the case study questions 

Q1. 1.Read and understand the case, then list all the Key Performance Indicators (KPIs) you will
 check for this data analysis (product-wise and team-wise).

In [9]:
product_group = df.groupby('product_code')

product_kpis = product_group.agg(
    total_revenue = ('product_amount_with_gst','sum'),
    transactions = ('product_amount_with_gst','count'),
    avg_order_value = ('product_amount_with_gst','mean'),
    unique_customers = ('user_id','nunique')
).reset_index()

user_txn_counts = df.groupby(['product_code','user_id']).size().reset_index(name='cnt')
repeat_customers = user_txn_counts[user_txn_counts['cnt']>1].groupby('product_code').size().reset_index(name='repeat_customers')
product_kpis = product_kpis.merge(repeat_customers, on='product_code', how='left')
product_kpis['repeat_customers'] = product_kpis['repeat_customers'].fillna(0).astype(int)
product_kpis['repeat_customer_rate_pct'] = (product_kpis['repeat_customers'] / product_kpis['transactions'].replace(0,np.nan) * 100).fillna(0)

ps = df.groupby('product_code').agg(
    success_txns = ('payment_status', lambda s: (s.str.lower()=='success').sum() ),
    coupon_used_txns = ('coupon_code', lambda s: (s != 'NoCoupon').sum() )
).reset_index()

df['lead_to_purchase_days'] = (df['sales_date'] - df['lead_registered_time']).dt.days
lead_median = df.groupby('product_code')['lead_to_purchase_days'].median().reset_index().rename(columns={'lead_to_purchase_days':'median_lead_to_purchase_days'})
ps = ps.merge(lead_median, on='product_code', how='left')

product_kpis = product_kpis.merge(ps, on='product_code', how='left')
product_kpis['payment_success_rate_pct'] = (product_kpis['success_txns'] / product_kpis['transactions'].replace(0,np.nan) * 100).fillna(0)
product_kpis['coupon_usage_rate_pct'] = (product_kpis['coupon_used_txns'] / product_kpis['transactions'].replace(0,np.nan) * 100).fillna(0)

prod_month = df.groupby(['product_code','sales_month'])['product_amount_with_gst'].sum().reset_index()
last_months = sorted(prod_month['sales_month'].unique())[-3:]
prod_mom = prod_month[prod_month['sales_month'].isin(last_months)].pivot(index='product_code', columns='sales_month', values='product_amount_with_gst').fillna(0)
prod_mom.columns = [f"rev_{c.strftime('%Y-%m')}" for c in prod_mom.columns]
product_kpis = product_kpis.merge(prod_mom.reset_index(), on='product_code', how='left')


product_kpis.to_csv(os.path.join(outdir, "product_kpis.csv"), index=False)


teams = ['Sales','Marketing','Product_Development','Branding','Customer_Support']
unique_products = sorted(df['product_code'].unique())
product_team_map = {p: teams[i % len(teams)] for i,p in enumerate(unique_products)}
df['team_owner'] = df['product_code'].map(product_team_map)

team_group = df.groupby('team_owner')
team_kpis = team_group.agg(
    total_revenue = ('product_amount_with_gst','sum'),
    transactions = ('product_amount_with_gst','count'),
    avg_order_value = ('product_amount_with_gst','mean'),
    unique_customers = ('user_id','nunique')
).reset_index()

user_counts_team = df.groupby(['team_owner','user_id']).size().reset_index(name='cnt')
repeat_team = user_counts_team[user_counts_team['cnt']>1].groupby('team_owner').size().reset_index(name='repeat_customers')
team_kpis = team_kpis.merge(repeat_team, on='team_owner', how='left')
team_kpis['repeat_customers'] = team_kpis['repeat_customers'].fillna(0).astype(int)
team_kpis['repeat_customer_rate_pct'] = (team_kpis['repeat_customers'] / team_kpis['transactions'].replace(0,np.nan) * 100).fillna(0)

lead_team = df.groupby('team_owner')['lead_to_purchase_days'].median().reset_index().rename(columns={'lead_to_purchase_days':'median_lead_to_purchase_days'})
team_kpis = team_kpis.merge(lead_team, on='team_owner', how='left')

team_ps = df.groupby('team_owner').agg(
    success_txns = ('payment_status', lambda s: (s.str.lower()=='success').sum() ),
    coupon_used_txns = ('coupon_code', lambda s: (s != 'NoCoupon').sum() )
).reset_index()
team_kpis = team_kpis.merge(team_ps, on='team_owner', how='left')
team_kpis['payment_success_rate_pct'] = (team_kpis['success_txns'] / team_kpis['transactions'].replace(0,np.nan) * 100).fillna(0)
team_kpis['coupon_usage_rate_pct'] = (team_kpis['coupon_used_txns'] / team_kpis['transactions'].replace(0,np.nan) * 100).fillna(0)

team_kpis.to_csv(os.path.join(outdir, "team_kpis.csv"), index=False)

print("Product KPIs (top 10 by revenue)")
display_cols = ['product_code','total_revenue','transactions','avg_order_value','unique_customers','repeat_customer_rate_pct','payment_success_rate_pct','coupon_usage_rate_pct','median_lead_to_purchase_days']
print(product_kpis.sort_values('total_revenue', ascending=False)[display_cols].head(10).to_string(index=False))

print("\n Team KPIs")
print(team_kpis.sort_values('total_revenue', ascending=False).to_string(index=False))

Product KPIs (top 10 by revenue)
product_code  total_revenue  transactions  avg_order_value  unique_customers  repeat_customer_rate_pct  payment_success_rate_pct  coupon_usage_rate_pct  median_lead_to_purchase_days
    Product1      3695029.2          1272      2904.897170               725                 17.216981                       0.0              25.864780                          -1.0
    Product4      2035095.0          1020      1995.191176               842                 11.764706                       0.0               0.000000                          -1.0
    Product3      1617387.0           838      1930.056086               541                 19.093079                       0.0               0.477327                          -1.0
    Product8      1411420.0          1560       904.756410              1029                 18.846154                       0.0               4.743590                          -1.0
    Product5       944046.0           504      1873.10714

Q2.If you’re conducting employee-level analysis for each team, what factors and data points? would you check to grade their performance?

In [11]:
if 'employee_id' in df.columns:
    emp_group = df.groupby('employee_id').agg(
        total_revenue = ('product_amount_with_gst','sum'),
        transactions = ('product_amount_with_gst','count'),
        avg_order_value = ('product_amount_with_gst','mean'),
        unique_customers = ('user_id','nunique'),
        success_txns = ('payment_status', lambda s: (s.str.lower()=='success').sum())
    ).reset_index()
    emp_group['payment_success_rate_pct'] = (emp_group['success_txns'] / emp_group['transactions'].replace(0,np.nan) * 100).fillna(0)
else:
    np.random.seed(2025)
    emp_rows = []
    for team in team_kpis['team_owner']:
        trev = float(team_kpis.loc[team_kpis['team_owner']==team,'total_revenue'].values[0])
        ttx = int(team_kpis.loc[team_kpis['team_owner']==team,'transactions'].values[0])
        weights = np.random.rand(5)
        weights = weights / weights.sum()
        for i,w in enumerate(weights, start=1):
            emp_id = f"{team[:3].upper()}_E{i:02d}"
            emp_revenue = round(trev * w,2)
            emp_txn = int(round(ttx * w)) if ttx>0 else 0
            avg_response_time_hours = round(np.random.uniform(2,72) if team=='Customer_Support' else np.random.uniform(8,240),1)
            emp_rows.append({
                'employee_id': emp_id,
                'team': team,
                'total_revenue': emp_revenue,
                'transactions': emp_txn,
                'avg_order_value': round(emp_revenue / emp_txn,2) if emp_txn>0 else 0,
                'avg_response_time_hours': avg_response_time_hours
            })
    emp_group = pd.DataFrame(emp_rows)


emp = emp_group.copy()
emp = emp.merge(team_kpis[['team_owner','repeat_customer_rate_pct']], left_on='team', right_on='team_owner', how='left').drop(columns=['team_owner'])
def minmax(series):
    if series.max()==series.min():
        return series.apply(lambda x: 0.5)
    return (series - series.min()) / (series.max() - series.min())
emp['s_revenue'] = minmax(emp['total_revenue'])
emp['s_transactions'] = minmax(emp['transactions'])
emp['s_aov'] = minmax(emp['avg_order_value'])
emp['s_repeat_rate'] = minmax(emp['repeat_customer_rate_pct'].fillna(0))
emp['s_response_time'] = 1 - minmax(emp['avg_response_time_hours'])

w_rev, w_txn, w_aov, w_repeat, w_resp = 0.40, 0.15, 0.15, 0.10, 0.20
emp['overall_score'] = emp['s_revenue']*w_rev + emp['s_transactions']*w_txn + emp['s_aov']*w_aov + emp['s_repeat_rate']*w_repeat + emp['s_response_time']*w_resp
def grade_from_score(s):
    if s >= 0.80: return 'A'
    if s >= 0.60: return 'B'
    if s >= 0.40: return 'C'
    return 'D'
emp['grade'] = emp['overall_score'].apply(grade_from_score)
emp[['employee_id','team','total_revenue','transactions','avg_order_value','avg_response_time_hours','overall_score','grade']].to_csv(os.path.join(outdir,'employee_scores.csv'), index=False)

print("\n Employee scoring snapshot ")
print(emp.sort_values('overall_score', ascending=False)[['employee_id','team','total_revenue','transactions','avg_order_value','avg_response_time_hours','overall_score','grade']].head(10).to_string(index=False))


 Employee scoring snapshot 
employee_id             team  total_revenue  transactions  avg_order_value  avg_response_time_hours  overall_score grade
    SAL_E04            Sales     1641512.17           814          2016.60                     32.4       0.816459     A
    CUS_E04 Customer_Support     1323852.12          1174          1127.64                     23.0       0.773601     B
    CUS_E02 Customer_Support     1378220.28          1223          1126.92                     44.8       0.772574     B
    SAL_E03            Sales     1468440.25           728          2017.09                     86.9       0.715627     B
    SAL_E01            Sales     1701028.95           844          2015.44                    204.7       0.677616     B
    CUS_E01 Customer_Support      783180.61           695          1126.88                     22.5       0.587414     C
    SAL_E02            Sales      914719.37           454          2014.80                     72.2       0.564294     C
   

Q3.How would you handle Root Cause Analysis (RCA) as a Data Analyst? Provide a short scenario based on your knowledge and explain the steps involved

In [12]:
monthly = df.groupby('sales_month').agg(
    revenue=('product_amount_with_gst','sum'),
    total_txns=('product_amount_with_gst','count'),
    coupon_used = ('coupon_code', lambda s: (s != 'NoCoupon').sum() ),
    direct_source_txns = ('source', lambda s: (s=='Direct').sum() ),
    failed_payments = ('payment_status', lambda s: (s.str.lower()!='success').sum() )
).reset_index().sort_values('sales_month')

monthly['revenue_change_pct'] = monthly['revenue'].pct_change().fillna(0) * 100
corr_candidates = monthly[['revenue','total_txns','coupon_used','direct_source_txns','failed_payments']]
corr_matrix = corr_candidates.corr()

rca_rows = []
for i in range(1, len(monthly)):
    rev_pct = monthly.loc[i,'revenue_change_pct']
    if rev_pct < -10:
        month = monthly.loc[i,'sales_month'].strftime('%Y-%m')
        reasons = []
        prev_direct = monthly.loc[i-1,'direct_source_txns']
        curr_direct = monthly.loc[i,'direct_source_txns']
        if prev_direct>0 and (curr_direct - prev_direct)/prev_direct < -0.10:
            reasons.append('Drop in Direct channel transactions (sales/channel)')
        if monthly.loc[i,'failed_payments'] > monthly.loc[i-1,'failed_payments']:
            reasons.append('Increase in failed/unsuccessful payments (checkout friction)')
        if monthly.loc[i,'coupon_used'] < monthly.loc[i-1,'coupon_used']:
            reasons.append('Drop in coupon/promo usage (marketing promo reduction)')
        if not reasons:
            reasons.append('No single dominant signal - suggests deeper analysis needed (pricing, competition, product issues)')
        rca_rows.append({
            'month': month,
            'revenue_change_pct': round(rev_pct,2),
            'candidate_reasons': '; '.join(reasons)
        })
rca_df = pd.DataFrame(rca_rows)

monthly.to_csv(os.path.join(outdir,'monthly_aggregates.csv'), index=False)
corr_matrix.to_csv(os.path.join(outdir,'monthly_correlation_matrix.csv'), index=False)
rca_df.to_csv(os.path.join(outdir,'rca_summary.csv'), index=False)

print("\n Monthly aggregates ")
print(monthly.to_string(index=False))

print("\n Correlation matrix (monthly signals) ")
print(corr_matrix.to_string())

if rca_df.empty:
    print("\nNo month showed >10% revenue decline based on monthly aggregates; RCA rule-based list is empty.")
else:
    print("\n RCA flagged months and candidate reasons ")
    print(rca_df.to_string(index=False))

# plots (matplotlib only)
plt.figure(figsize=(8,4))
plt.plot(monthly['sales_month'].dt.strftime('%Y-%m'), monthly['revenue'], marker='o')
plt.title('Monthly Revenue (for RCA view)')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(outdir,'rca_monthly_revenue.png'))
plt.close()

plt.figure(figsize=(8,4))
plt.plot(monthly['sales_month'].dt.strftime('%Y-%m'), monthly['failed_payments'], marker='o')
plt.title('Monthly Failed Payments')
plt.xlabel('Month')
plt.ylabel('Failed Payments Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(outdir,'rca_failed_payments.png'))
plt.close()




 Monthly aggregates 
sales_month    revenue  total_txns  coupon_used  direct_source_txns  failed_payments  revenue_change_pct
 2022-06-01 5788559.64        4432          332                3017             4432            0.000000
 2022-07-01 8147819.20        5082          330                4023             5082           40.757282

 Correlation matrix (monthly signals) 
                    revenue  total_txns  coupon_used  direct_source_txns  failed_payments
revenue                 1.0         1.0         -1.0                 1.0              1.0
total_txns              1.0         1.0         -1.0                 1.0              1.0
coupon_used            -1.0        -1.0          1.0                -1.0             -1.0
direct_source_txns      1.0         1.0         -1.0                 1.0              1.0
failed_payments         1.0         1.0         -1.0                 1.0              1.0

No month showed >10% revenue decline based on monthly aggregates; RCA rule-based l